<a href="https://colab.research.google.com/github/jjk767557/stable-diffusion/blob/main/stable_diffusion_interactive_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Interactive Notebook 📓 🤖

A widgets-based interactive notebook for Google Colab that lets users generate AI images from prompts (Text2Image) using [Stable Diffusion (by Stability AI, Runway & CompVis)](https://en.wikipedia.org/wiki/Stable_Diffusion).

This notebook aims to be an alternative to WebUIs while offering a simple and lightweight GUI for anyone to get started with Stable Diffusion.

Uses Stable Diffusion, [HuggingFace](https://huggingface.co/) Diffusers and [Jupyter widgets](https://github.com/jupyter-widgets/ipywidgets).

<br/>

Made with ❤️ by redromnon

[GitHub](https://github.com/redromnon/stable-diffusion-interactive-notebook)

In [ ]:
#@title 👇 Installing dependencies { display-mode: "form" }
#@markdown ---
#@markdown Make sure to select **GPU** as the runtime type:<br/>
#@markdown *Runtime->Change Runtime Type->Under Hardware accelerator, select GPU*
#@markdown
#@markdown ---

!pip -q install torch diffusers transformers accelerate scipy safetensors xformers mediapy ipywidgets==7.7.1

In [ ]:
#@title 👇 Selecting Model { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Select Model** - A list of Stable Diffusion models to choose from.
#@markdown - **Select Sampler** - A list of schedulers to choose from. Default is EulerAncestralScheduler.
#@markdown - **Safety Checker** - Enable/Disable uncensored content
#@markdown
#@markdown ---

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, DDIMScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib

#Enable third party widget support
from google.colab import output
output.enable_custom_widget_manager()

#Pipe
pipe = None

#Models
select_model = widgets.Dropdown(
    options=[
        ("Stable Diffusion 2.1 Base" , "stabilityai/stable-diffusion-2-1-base"),
        ("Stable Diffusion 2.1" , "stabilityai/stable-diffusion-2-1"),
        ("Stable Diffusion 1.5", "runwayml/stable-diffusion-v1-5"),
        ("Dreamlike Photoreal 2.0" , "dreamlike-art/dreamlike-photoreal-2.0"),
        ("OpenJourney v4" , "prompthero/openjourney-v4")
    ],
    description="Select Model:"
)

#Schedulers
select_sampler = widgets.Dropdown(
    options=[
        "EulerAncestralDiscreteScheduler",
        "EulerDiscreteScheduler",
        "UniPCMultistepScheduler",
        "DDIMScheduler"
    ],
    description="Select Schedular:"
)
select_sampler.style.description_width = "auto"

#Safety Checker
safety_check = widgets.Checkbox(
    value=True,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)


#Get scheduler
def get_scheduler(name):

  match name:

    case "EulerAncestralDiscreteScheduler":
      return EulerAncestralDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "DDIMScheduler":
      return DDIMScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "EulerDiscreteScheduler":
      return EulerDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "UniPCMultistepScheduler":
      return UniPCMultistepScheduler.from_pretrained(select_model.value, subfolder="scheduler")

#Run pipeline
def pipeline(p):

  global pipe

  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running, please wait...")

    pipe = StableDiffusionPipeline.from_pretrained(
      select_model.value,
      scheduler=get_scheduler(select_sampler.value),
      torch_dtype=torch.float16,
      vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
    ).to("cuda")

    if not safety_check.value:
      pipe.safety_checker = None

    pipe.enable_xformers_memory_efficient_attention()

    print("Finished!")

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"),
      select_model, select_sampler, safety_check, apply_btn, out
    ]
)


In [ ]:
# 安裝必要的依賴
!pip install diffusers transformers accelerate torch pillow ipywidgets

from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

# 加載 Stable Diffusion 圖生圖模型
print("正在加載 Stable Diffusion 模型...")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16
).to("cuda")
print("模型加載成功！")

# 全局變量存儲圖片路徑
uploaded_base_image_path = None
uploaded_target_image_path = None
last_generated_image_path = None

# 創建介面組件
upload_base_button = widgets.Button(description="上傳量體圖片")
upload_target_button = widgets.Button(description="上傳目標圖片")
prompt_box = widgets.Text(
    value='A photorealistic Catholic church with modern design and sacred elements.',
    placeholder='請輸入您的 Prompt（描述）',
    description='Prompt:',
    layout=widgets.Layout(width='70%')
)
negative_prompt_box = widgets.Text(
    value='low-quality rendering, unrealistic textures, abstract design',
    placeholder='請輸入您的 Negative Prompt（排除內容）',
    description='Negative Prompt:',
    layout=widgets.Layout(width='70%')
)
strength_slider = widgets.FloatSlider(
    value=0.2,
    min=0.1,
    max=0.5,
    step=0.05,
    description='Strength (Base Influence):',
    layout=widgets.Layout(width='50%')
)
steps_slider = widgets.IntSlider(
    value=100,
    min=50,
    max=150,
    step=10,
    description='Steps:',
    layout=widgets.Layout(width='50%')
)
cfg_slider = widgets.FloatSlider(
    value=12.0,
    min=7.0,
    max=20.0,
    step=0.5,
    description='CFG Scale:',
    layout=widgets.Layout(width='50%')
)
generate_button = widgets.Button(description="開始生成")
output = widgets.Output()

# 定義上傳量體圖片功能
def on_upload_base_clicked(b):
    global uploaded_base_image_path
    with output:
        output.clear_output()
        print("請選擇量體圖片...")
        uploaded = files.upload()
        if uploaded:
            uploaded_base_image_path = list(uploaded.keys())[0]
            print(f"量體圖片已成功上傳：{uploaded_base_image_path}")

# 定義上傳目標圖片功能
def on_upload_target_clicked(b):
    global uploaded_target_image_path
    with output:
        output.clear_output()
        print("請選擇目標圖片...")
        uploaded = files.upload()
        if uploaded:
            uploaded_target_image_path = list(uploaded.keys())[0]
            print(f"目標圖片已成功上傳：{uploaded_target_image_path}")

# 定義生成圖像功能
def on_generate_clicked(b):
    global uploaded_base_image_path, uploaded_target_image_path, last_generated_image_path
    with output:
        output.clear_output()
        if not uploaded_base_image_path or not uploaded_target_image_path:
            print("請先上傳量體圖片和目標圖片！")
            return
        try:
            # 加載量體圖片
            base_image = Image.open(uploaded_base_image_path).convert("RGB").resize((512, 512))
            # 加載目標圖片（僅作為參考）
            target_image = Image.open(uploaded_target_image_path).convert("RGB").resize((512, 512))

            print(f"量體圖片已加載：{uploaded_base_image_path}")
            print(f"目標圖片已加載：{uploaded_target_image_path}")
        except FileNotFoundError as e:
            print(f"錯誤：找不到圖片文件 - {str(e)}")
            return

        # 使用 Prompt 生成圖像
        prompt = prompt_box.value
        negative_prompt = negative_prompt_box.value
        strength = strength_slider.value
        steps = steps_slider.value
        guidance_scale = cfg_slider.value

        print(f"提示詞：{prompt}")
        print(f"排除提示詞：{negative_prompt}")
        print(f"Strength 值：{strength}, Steps：{steps}, CFG Scale：{guidance_scale}")

        print("生成圖像中，請稍候...")
        images = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=base_image,
            strength=strength,
            guidance_scale=guidance_scale,
            num_inference_steps=steps
        ).images

        # 顯示生成的圖像
        images[0].show()

        # 保存生成的圖像
        last_generated_image_path = "/content/generated_image.jpg"
        images[0].save(last_generated_image_path)
        print(f"生成的圖像已保存至：{last_generated_image_path}")

# 綁定按鈕事件
upload_base_button.on_click(on_upload_base_clicked)
upload_target_button.on_click(on_upload_target_clicked)
generate_button.on_click(on_generate_clicked)

# 顯示介面
display(widgets.VBox([
    widgets.HBox([upload_base_button, upload_target_button, generate_button]),
    prompt_box,
    negative_prompt_box,
    widgets.HBox([strength_slider, steps_slider, cfg_slider]),
    output
]))
